<b>I. Installation des packages necessaires</b>



In [10]:
!pip install simplemma
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<b>II. Import des librairies</b> 

In [11]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function

#librairies usuelles
import pandas as pd
import numpy as np 
from collections import Counter 
import itertools

#librairies pour le traitement automatique du language naturel
import nltk
from nltk.tokenize import word_tokenize
import simplemma 
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

#librairies pour la génératon de la matrice tf-idf du corpus du champs "DECRIPTION" 
from sklearn.feature_extraction.text import TfidfVectorizer

#librairies pour le clustering 
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import DBSCAN


<b>III. Chargement du dataframe et suppression des colonnes qui ne contiennent aucune valeur</b>



In [12]:
#Chemin du dataframe
file_path="Dataset - Ads _ Levallois-Perret - 2019-08 - export-ads-levallois-perret-2019-08-27.tsv"
df=pd.read_csv(file_path,sep='\t')
#Recuperation des colonnnes qui ne contiennent aucune valeur
cols_empty = [col for col in df.columns if df[col].isnull().all()]
#Suppresssion des colonnes qui ne contiennent aucune valeur
df.drop(cols_empty,axis=1,inplace=True)
print(df)

                                        ID  ... LAST_PRICE_DECREASE_DATE
0     22c05930-0eb5-11e7-b53d-bbead8ba43fe  ...                      NaN
1     8d092fa0-bb99-11e8-a7c9-852783b5a69d  ...               2018-09-25
2     44b6a5c0-3466-11e9-8213-25cc7d9bf5fc  ...                      NaN
3     e9e07ed0-812f-11e8-82aa-61eacebe4584  ...                      NaN
4     872302b0-5a21-11e9-950c-510fefc1ed35  ...               2019-06-14
...                                    ...  ...                      ...
2159  d3579370-824f-11e9-af18-9742751bcff8  ...               2019-07-07
2160  cce3fc60-c86b-11e9-a6b2-651beb16710e  ...                      NaN
2161  beec50b0-c85e-11e9-92d3-cb429fb9e457  ...                      NaN
2162  8cba88c0-a07a-11e9-a8e6-0de7b497e456  ...                      NaN
2163  f3ae8be0-9fcf-11e9-ab3e-47ec2b68d334  ...                      NaN

[2164 rows x 46 columns]


<b>IV. Vectorisation des textes contenus dans le champs "DESCRIPTION" du dataframe</b>

Les differents étapes réalisées sont : 

1. Preprocessing

Le preprocessing du texte contenu dans le champs "DESCRIPTION" comporte les étapes suivantes : 
  - Transformation du texte en minuscules
  - Tokenization des phrases en des mots et formation de la liste qui contient tout les mots de ce corpus 
  -  Lemmatization des mots : operation qui associe à un ensemble de mots une forme neutre
  
  EX: voiture = voitures, voiture
  - On ne garde pas les mots qui comporte une lettre sauf les chiffres : permet de retirer la pontuation dans les textes

2. Extraction des features pertinents

Les features pertinents c'est les 500 mots les plus fréquents de la liste des mots construite précedemment 

3. Vectorisation des textes contenus dans le champs "DESCRIPTION"

On vectorise le texte correspondant à chaque bien immobilier dans l'espace des features pertinents en utilisant la métrique tf-idf.

Une documentation vers cette métrique : https://fr.wikipedia.org/wiki/TF-IDF

<b>V. Clustering des vecteurs et detection des doublons</b> 

Les vecteurs determinés sont caracteristiques des textes contenus dans les champs "DESCRIPTION" representant chaque bien immobilier. 

On utilisera l'<b><u>algorithme de clustering par propagation d'affinités</u></b>. C'est une excellent algorithme dans les espaces de très grande dimension (pour notre cas la dimension = 500). L'autre avantage de cet algorithme est que le nombre 'k' de clusters est determiné automatiquement en fonction des zonnes de haute variations.

<b><u>Un  cluster qui contiennent plus d'un élement correspond à un bien immobilier en doublons</b></u>

Pour l'instant on ne traite que les informations contenus dans le champs "DESCRIPTION" du dataframe. Par la suite nous ferrons comment ammeliorer la detection des doublons en utilisant les informations des autres colonnes du dataframe autre que celui du champs "DESCRIPTION".

In [7]:
french_stop_words=nltk.corpus.stopwords.words('french')
digit_=[str(x) for x in range(10)]

def tt(x):
	if x in french_stop_words:
		return False
	elif len(x)==1 and x not in digit_:
		return False
	else:
		return True

#Tokenisation de la langue française
tokenizer = nltk.data.load('tokenizers/punkt/PY3/french.pickle')
#Pour la lemmatisation en langue française
langdata=langdata = simplemma.load_data('fr')

#Preprocessing et recuperation du vocabulaire (mots)
wds=[]
corpora_description=[]
features_description=[]
for i in range(len(df)):
	#tokenize in sentences
	sentences=tokenizer.tokenize(str(df['DESCRIPTION'][i]))
	description_i=""
	for sen in sentences:
		#tokenize in words
		wds_sen=[x.lower() for x in word_tokenize(sen, language='french') if tt(x) and x!='..']
		#Lemnization
		wds_lematize=[simplemma.lemmatize(t, langdata) for t in wds_sen]
		#
		wds=wds+wds_lematize
		description_i=description_i+ " ".join(wds_lematize)

	corpora_description.append(description_i)

wds_freq=Counter(wds)
wds_freq=dict(sorted(wds_freq.items(),reverse=True,key=lambda t: t[1]) )# tri par ordre de frequence decroissant
wds_freq=dict(itertools.islice(wds_freq.items(), 500))#prends 500 termes les plus frequents 

#Recuperation des features
features_description=list(wds_freq.keys())
print("features : ")
print(features_description)

#Formation des vecteurs du corpus : vectors_corpora_description
vectorizer = TfidfVectorizer(vocabulary=features_description)#max_features=400,
vectors_corpora_description= vectorizer.fit_transform(corpora_description)
vectors_corpora_description=list(vectors_corpora_description.toarray())

#Clustering 
clustering = AffinityPropagation(max_iter=500,damping=0.8,verbose=True).fit(vectors_corpora_description)
labels_corpora_description=clustering.labels_

#Contruction de la 1ère collection des doublons
doublons_corpora_description={}
for i in range(len(vectors_corpora_description)):
	if labels_corpora_description[i] not in doublons_corpora_description.keys():
		doublons_corpora_description[labels_corpora_description[i]]=[[i],1]
	else:
		doublons_corpora_description[labels_corpora_description[i]][0].append(i)
		doublons_corpora_description[labels_corpora_description[i]][1]+=1


features : 
["d'une", 'salle', 'levallois', 'bien', 'appartement', 'charge', 'chambre', 'm²', 'cuisine', 'pièce', 'wc', 'honoraire', "d'un", 'immeuble', 'étage', '2', 'parking', 'le', 'séjour', 'situer', 'cave', 'euro', 'bain', 'entrée', '3', 'plus', 'un', 'rue', 'métro', 'prix', 'commerce', 'grand', 'tout', 'standing', 'ascenseur', 'très', 'ce', 'proposer', 'deux', 'balcon', 'proximité', 'vente', "d'eau", 'copropriété', 'dont', 'équipée', 'annonce', 'bureau', 'calme', 'ttc', '000', 'comprendre', '1', 'loyer', 'place', 'surface', 'beau', 'lumineux', 'aménager', 'annuel', 'choix', 'sous-sol', 'pied', 'résidence', 'sécuriser', 'à le', 'sans', '5', 'double', 'séparé', 'rangement', 'il', 'garantie', 'studio', 'état', 'bon', 'composer', 'proche', 'espace', 'partners', 'building', '4', 'de', 'terrer', 'france', 'référence', 'ligne', 'récent', 'jardin', 'dans', 'jour', 'bel', 'michel', 'louise', 'anatole', 'm2', 'entièrement', 'séparer', '92300', 'chauffage', 'offre', 'indépendant', 'mois', '

<b>V. Determination de la nouvelle collection de doublons en utilisant les informations contenus dans les autres colonnes du dataframe</b>

L'incovenient de ce limiter à l'approche précédente est que deux biens immobiliers peuvent avoir le même texte dans le champs "DESCRIPTION" mais ils ne sont pas identiques. 

Pour resoudre ce problème, pour chaque cluster determiner précèdemment, on refait un micro-clustering en utilisant les features des colonnes : 'PROPERTY_TYPE', 'NEW_BUILD','SURFACE','ROOM_COUNT','PARKING','PRICE','PRICE_M2','RENTAL_EXPENSES_INCLUDED','DEALER_NAME' 'DEALER_TYPE'. 

Nous avons remplacer les champs vides d'une colonne par la moyenne des champs qui la constituent

La detection de doublons est maintenant plus pertinante. 

<b><u>Résultats : </b></u> 
- Nombre des biens immobiliers qui apparaissent en doublons : 357
- Nombre des biens immobiliers qui n'apparaissent pas en doublons : 959

In [8]:
#Nouvelles collection de doublons
doublons_new={}
df_sub=df[['PROPERTY_TYPE', 'NEW_BUILD','SURFACE','ROOM_COUNT','PARKING','PRICE','PRICE_M2','RENTAL_EXPENSES_INCLUDED','DEALER_NAME','DEALER_TYPE']]
nb_cluster=len(doublons_corpora_description.keys())
#Transforme les chaines de caracteère en valeurs numeriques
labelencoder=LabelEncoder()
#var_x=nb_cluster
var_x=-1
nb_biens_imobiliers_en_doublons=0
for k,v in doublons_corpora_description.items():
  vectors_sub_id_cluster=df_sub.iloc[v[0],:]
  #Remplacement par la moyenne : colonnes numeriques
  vectors_sub_id_cluster[['SURFACE','ROOM_COUNT','PRICE','PRICE_M2']].fillna(value=vectors_sub_id_cluster[['SURFACE','ROOM_COUNT','PRICE','PRICE_M2']].mean(), inplace=True)
  #Remplacement par le champs '' : colonnes qui contiennet du texte
  vectors_sub_id_cluster.fillna('',inplace=True)
  for col in vectors_sub_id_cluster.columns:
    #print(vectors_sub_id_cluster[col])
    vectors_sub_id_cluster[col]=labelencoder.fit_transform(vectors_sub_id_cluster[col].astype(str))

  clustering_sub = DBSCAN(eps=1.5, min_samples=2,n_jobs=-1).fit(vectors_sub_id_cluster)
  labels_sub=clustering_sub.labels_		
  dico={}
  for i in range(len(labels_sub)):
    if labels_sub[i] not in dico.keys():
      dico[labels_sub[i]]=[[v[0][i]],1]
    else:
      dico[labels_sub[i]][0].append(v[0][i])
      dico[labels_sub[i]][1]+=1
  #Rajout
  for kk in dico.keys():
    if kk==-1: #les elements avec un id_cluster=-1 n'appartiennent a aucun clusters : ce sont les bien impbiliers qui apparaissent sans doublons
      if -1 not in doublons_new.keys():
        doublons_new[-1]=dico[kk]
      else:
        doublons_new[-1][0]+=dico[kk][0]
        doublons_new[-1][1]+=dico[kk][1]
    else:
      var_x+=1
      doublons_new[var_x]=dico[kk]
      nb_biens_imobiliers_en_doublons+=1

print("nombre de biens immobiliers qui n'apparaissent pas en doublons : ")
print(len(doublons_new[-1][0]))
print(len(doublons_corpora_description.keys()))
print('nombre de biens immobiliers qui apparaissent en doublons : ')
print(nb_biens_imobiliers_en_doublons)


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local

nombre de biens immobiliers qui n'apparaissent pas en doublons : 
959
328
nombre de biens immobiliers qui apparaissent en doublons : 
357


<b>VI. Data Viz : Géneration des fichiers csv et interpretation des résultats </b>

Pour finir nous avons générer deux fichiers csv (séparateur : tabulation ('\t') ) :
1. "doublons.csv"

Contient tous les biens immobiliers en doublons. 

Un ensemble de biens immobliers en doublons est séparé par une ligne dans le fichier.
Et nous affichons succesivement les doublons de 2,3,4,5,6,etc.

2. "sans_doublons.csv"

Contient tous les biens immobiliers qui ne sont pas en doublons

In [9]:

#Tri par ordre croissant
doublons_new=dict(sorted(doublons_new.items(),key=lambda t: t[1][1]))

columns_good=['ID','DESCRIPTION','CRAWL_SOURCE','PROPERTY_TYPE','SURFACE','ROOM_COUNT','PRICE','PRICE_M2','DEALER_NAME','DEALER_TYPE']
columns_others=[str(x) for x in df.columns if x not in columns_good]
columns=columns_good+columns_others
#Génération de fichier csv pour DataViZ
df=df.fillna('')
df = df.applymap(str)
with open('doublons.csv','w+',encoding="utf-8") as f, open("sans_doublons.csv", 'w+',encoding="utf-8") as f_s:
  f.write("ID_CLUSTER")
  f_s.write("ID_CLUSTER")
  for i in range(len(columns)):
    f.write('\t'+columns[i])
    f_s.write('\t'+columns[i])
    if i==len(columns)-1:
      f.write('\n')
      f_s.write('\n')
  for k,v in doublons_new.items():
    #Ecriture des biens immobiliers qui n'apparaissent pas en doublons
    if v[1]==1 or k==-1:
      for idd in v[0]:
        f_s.write(str(k))
        for i in range(len(columns)):
          f_s.write('\t'+df[columns[i]][idd])
          if i==len(columns)-1:
            f_s.write('\n')
    #Ecriture des biens immobiliers qui apparaissent en doublons
    else:
      for idd in v[0]:
        f.write(str(k))
        for i in range(len(columns)):
          f.write('\t'+df[columns[i]][idd])
          if i==len(columns)-1:
            f.write('\n')
      for i in range(len(columns)):
        if i<len(columns)-1:
          f.write(''+'\t')
        else :
          f.write(''+'\n')

